# Group Classification (ZenML)

Train the `GroupClassifier` on parquet-based time-group data:
- Load groups from `ml_output_*.parquet` using the **C++ Arrow dataloader** (zero-copy)
- Train via the new ZenML `group_classification_pipeline`
- Export a TorchScript model (for C++/Python inference)

**Note:** the model predicts a label *per time-group* (not per event). We aggregate to events later via the output adapter.


In [1]:
from pioneerml.zenml import utils as zenml_utils

PROJECT_ROOT = zenml_utils.find_project_root()
zenml_client = zenml_utils.setup_zenml_for_notebook(root_path=PROJECT_ROOT, use_in_memory=True)
print(f"ZenML ready with stack: {zenml_client.active_stack_model.name}")


Using ZenML repository root: /workspace
Ensure this is the top-level of your repo (.zen must live here).
Initializing the ZenML global configuration version to 0.92.0
Creating database tables
Creating default project 'default' ...
Creating default stack...
The current repo active project is no longer available.
Setting the repo active project to 'default'.
The current repo active stack is no longer available. Resetting the active stack to default.
Setting the global active project to 'default'.
Setting the global active stack to default.
Reloading configuration file /workspace/.zen/config.yaml
ZenML ready with stack: default


In [2]:
from pathlib import Path
from datetime import datetime
import sys

import pyarrow.parquet as pq
import torch

from pioneerml.zenml import load_step_output

# Ensure the C++ dataloader Python bindings are on the path
pml_bindings = Path(PROJECT_ROOT) / "external" / "pioneerml_dataloaders" / "build" / "bindings"
sys.path.insert(0, str(pml_bindings))

from pioneerml.zenml.pipelines.training import group_classification_pipeline


In [3]:
# Build a tiny parquet shard for quick debugging

data_dir = Path(PROJECT_ROOT) / "data"
src_parquet = data_dir / "ml_output_000.parquet"

small_parquet = Path(PROJECT_ROOT) / "artifacts" / "classify_groups_small.parquet"
small_rows = 8

table = pq.read_table(src_parquet)
table = table.slice(0, small_rows)
small_parquet.parent.mkdir(parents=True, exist_ok=True)
pq.write_table(table, small_parquet)

parquet_paths = [str(small_parquet)]
print(f"Wrote small parquet: {small_parquet} (rows={small_rows})")


Wrote small parquet: /workspace/artifacts/classify_groups_small.parquet (rows=8)


In [4]:
# Run the new ZenML pipeline

run = group_classification_pipeline.with_options(enable_cache=False)(
    parquet_paths=parquet_paths,
    config_json={"time_window_ns": 1.0},
    max_epochs=1,
    lr=1e-3,
    weight_decay=1e-4,
)
print(f"Run name: {run.name}")
print(f"Run status: {run.status}")


Initiating a new run for the pipeline: group_classification_pipeline.
Registered new pipeline: group_classification_pipeline.
Caching is disabled by default for group_classification_pipeline.
Using user: default
Using stack: default
  deployer: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step load_group_classifier_data has started.
Step load_group_classifier_data has finished in 0.164s.
Step train_group_classifier has started.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type                  ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model   │ GroupClassifierStereo │  1.5 M │ train │     0 │
│ 1 │ loss_fn │ BCEWithLogitsLoss     │      0 │ train │     0 │
└───┴─────────┴───────────────────────┴────────┴───────┴───────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 6                                                                          
Modules in train mode: 56                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.


Step train_group_classifier has finished in 2.098s.
Pipeline run has finished in 3.311s.
Run name: group_classification_pipeline-2026_02_01-03_14_06_198794
Run status: completed


In [5]:
# Load artifacts and run a quick forward pass

trained_module = load_step_output(run, "train_group_classifier")
batch = load_step_output(run, "load_group_classifier_data")

if trained_module is None or batch is None:
    raise RuntimeError("Could not load artifacts from the group_classification_pipeline run.")

trained_module.eval()
device = next(trained_module.parameters()).device
data = batch.data.to(device)

with torch.no_grad():
    logits = trained_module.model(data)

print(f"Device: {device}")
print(f"Events (graphs): {data.num_graphs}")
print(f"Groups: {data.num_groups}")
print(f"x: {tuple(data.x.shape)}")
print(f"edge_index: {tuple(data.edge_index.shape)}")
print(f"edge_attr: {tuple(data.edge_attr.shape)}")
print(f"group_ptr: {tuple(data.group_ptr.shape)}")
print(f"time_group_ids: {tuple(data.time_group_ids.shape)}")
print(f"logits: {tuple(logits.shape)}")


Device: cpu
Events (graphs): 8
Groups: 97
x: (416, 4)
edge_index: (2, 26378)
edge_attr: (26378, 4)
group_ptr: (9,)
time_group_ids: (416,)
logits: (97, 3)


In [7]:
# Export TorchScript model

export_dir = Path(PROJECT_ROOT) / "trained_models" / "groupclassifier"
export_dir.mkdir(parents=True, exist_ok=True)

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
torchscript_path = export_dir / f"groupclassifier_{stamp}_torchscript.pt"

trained_module.model.cpu()
example = batch.data
trained_module.model.export_torchscript(torchscript_path, example, strict=False)
print(f"Saved TorchScript model: {torchscript_path}")


Saved TorchScript model: /workspace/trained_models/groupclassifier/groupclassifier_20260201_031410_torchscript.pt


## Save the TorchScript Model

Export the trained model to TorchScript so it can be loaded in C++ or Python.


In [8]:
# (Optional) Inspect a few target labels

print("targets sample:", batch.targets[:5])


targets sample: tensor([[1., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]) tensor([[1., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])
